In [5]:
import sys
sys.path.append('../../../')

import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import subprocess

import dask.multiprocessing
dask.config.set(schedular='process')

from utils.cuda_cluster import *
from utils.dataset import read_data, factorize_small_cardinality


import core.config as conf


In [6]:
client = Client()
client

/home/nyongja/anaconda3/envs/dask/lib/python3.7/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34327 instead
  http_address["port"], self.http_server.port


Client Scheduler: tcp://127.0.0.1:37857 Dashboard: http://127.0.0.1:34327/status,Cluster Workers: 5 Cores: 20 Memory: 33.47 GB


In [7]:
data_path = conf.raw_data_path + 'part-00175'
df = read_data(data_path, n_partitions=conf.n_partitions)
features = [
    'text_tokens',    ###############
    'hashtags',       #Tweet Features
    'tweet_id',       #
    'media',          #
    'links',          #
    'domains',        #
    'tweet_type',     #
    'language',       #
    'timestamp',      ###############
    'creator_user_id',              ###########################
    'creator_follower_count',       #Engaged With User Features
    'creator_following_count',      #
    'creator_is_verified',          #
    'creator_account_creation',     ###########################
    'engager_user_id',              #######################
    'engager_follower_count',       #Engaging User Features
    'engager_following_count',      #
    'engager_is_verified',          #
    'engager_account_creation',     #######################
    'engager_follows_creator',    #################### Engagement Features
    'reply',          #Target Reply
    'retweet',        #Target Retweet    
    'retweet_comment',#Target Retweet with comment
    'like',           #Target Like
                      ####################
]

df.columns = features

df = df.drop('text_tokens', axis=1)
df, = dask.persist(df)
_ = wait(df)

df.head()

TypeError: read_data() got an unexpected keyword argument 'n_partitions'

In [ ]:
cluster = LocalCUDACluster()
client = Client(cluster)
client

In [ ]:
%%time

NP = 16
path = '/dataset/raw/'
df = dask_cudf.read_parquet(f'{path}/part-00000', sep='\x01', header=None)#, dtype=DTYPES)
df = df.repartition(npartitions=NP)
df, = dask.persist(df)
_ = wait(df)
print('number of rows:',len(df))